In [1]:
import findspark
findspark.init('C:\spark\spark')
from pyspark import SparkContext 
from pyspark.sql import SQLContext 
import pandas as pd 
from pyspark.mllib.stat import Statistics

In [2]:
sc = SparkContext('local','NIDS')
sqlc=SQLContext(sc) 
df=pd.read_csv(r'UNSW_NB15_training-set.csv') 
sdf=sqlc.createDataFrame(df)

In [3]:
sdf

DataFrame[id: bigint, dur: double, proto: string, service: string, state: string, spkts: bigint, dpkts: bigint, sbytes: bigint, dbytes: bigint, rate: double, sttl: bigint, dttl: bigint, sload: double, dload: double, sloss: bigint, dloss: bigint, sinpkt: double, dinpkt: double, sjit: double, djit: double, swin: bigint, stcpb: bigint, dtcpb: bigint, dwin: bigint, tcprtt: double, synack: double, ackdat: double, smean: bigint, dmean: bigint, trans_depth: bigint, response_body_len: bigint, ct_srv_src: bigint, ct_state_ttl: bigint, ct_dst_ltm: bigint, ct_src_dport_ltm: bigint, ct_dst_sport_ltm: bigint, ct_dst_src_ltm: bigint, is_ftp_login: bigint, ct_ftp_cmd: bigint, ct_flw_http_mthd: bigint, ct_src_ltm: bigint, ct_srv_dst: bigint, is_sm_ips_ports: bigint, attack_cat: string, label: bigint]

In [4]:
sdf.show()

+---+-------------------+-----+-------+-----+-----+-----+------+------+------------------+----+----+------------------+------------------+-----+-----+------------------+------------------+------------------+------------------+----+----------+----------+----+-------------------+--------------------+--------------------+-----+-----+-----------+-----------------+----------+------------+----------+----------------+----------------+--------------+------------+----------+----------------+----------+----------+---------------+----------+-----+
| id|                dur|proto|service|state|spkts|dpkts|sbytes|dbytes|              rate|sttl|dttl|             sload|             dload|sloss|dloss|            sinpkt|            dinpkt|              sjit|              djit|swin|     stcpb|     dtcpb|dwin|             tcprtt|              synack|              ackdat|smean|dmean|trans_depth|response_body_len|ct_srv_src|ct_state_ttl|ct_dst_ltm|ct_src_dport_ltm|ct_dst_sport_ltm|ct_dst_src_ltm|is_ftp_logi

In [5]:
type(sdf)

pyspark.sql.dataframe.DataFrame

In [6]:
sdf.printSchema()

root
 |-- id: long (nullable = true)
 |-- dur: double (nullable = true)
 |-- proto: string (nullable = true)
 |-- service: string (nullable = true)
 |-- state: string (nullable = true)
 |-- spkts: long (nullable = true)
 |-- dpkts: long (nullable = true)
 |-- sbytes: long (nullable = true)
 |-- dbytes: long (nullable = true)
 |-- rate: double (nullable = true)
 |-- sttl: long (nullable = true)
 |-- dttl: long (nullable = true)
 |-- sload: double (nullable = true)
 |-- dload: double (nullable = true)
 |-- sloss: long (nullable = true)
 |-- dloss: long (nullable = true)
 |-- sinpkt: double (nullable = true)
 |-- dinpkt: double (nullable = true)
 |-- sjit: double (nullable = true)
 |-- djit: double (nullable = true)
 |-- swin: long (nullable = true)
 |-- stcpb: long (nullable = true)
 |-- dtcpb: long (nullable = true)
 |-- dwin: long (nullable = true)
 |-- tcprtt: double (nullable = true)
 |-- synack: double (nullable = true)
 |-- ackdat: double (nullable = true)
 |-- smean: long (nullabl

In [7]:
sdf.describe('label','synack','dbytes','Sttl','ct_srv_src','Dttl').show()

+-------+------------------+--------------------+------------------+------------------+------------------+------------------+
|summary|             label|              synack|            dbytes|              Sttl|        ct_srv_src|              Dttl|
+-------+------------------+--------------------+------------------+------------------+------------------+------------------+
|  count|            175341|              175341|            175341|            175341|            175341|            175341|
|   mean|0.6806223302022916|0.021020450476500106|14928.918564397374|  179.546996994428| 9.306437170998226| 79.60956650184498|
| stddev|0.4662368642329709| 0.04339978350809942|143654.21771787005|102.94001105701703|10.704331220592314|110.50686317659928|
|    min|                 0|                 0.0|                 0|                 0|                 1|                 0|
|    max|                 1|            2.100352|          14655550|               255|                63|            

In [9]:
col_fea = sdf.select('label','synack','dbytes','Sttl','ct_srv_src','Dttl')
df_rdd = col_fea.rdd.map(lambda row: row[0:])
_corr = Statistics.corr(df_rdd,method = 'pearson')
print(_corr)

[[ 1.00000000e+00  5.82989292e-02 -7.68708606e-02  6.92741441e-01
   2.29043930e-01  9.50486050e-02]
 [ 5.82989292e-02  1.00000000e+00  1.01448286e-04  4.25897581e-02
  -2.76274751e-01  7.46999694e-01]
 [-7.68708606e-02  1.01448286e-04  1.00000000e+00 -1.35515187e-01
  -4.55293169e-02  2.35588727e-02]
 [ 6.92741441e-01  4.25897581e-02 -1.35515187e-01  1.00000000e+00
   3.46079095e-01 -3.28233995e-02]
 [ 2.29043930e-01 -2.76274751e-01 -4.55293169e-02  3.46079095e-01
   1.00000000e+00 -4.17337850e-01]
 [ 9.50486050e-02  7.46999694e-01  2.35588727e-02 -3.28233995e-02
  -4.17337850e-01  1.00000000e+00]]


In [10]:
from pyspark.sql.functions import corr

In [11]:
sdf.select(corr('label','synack')).show()

+-------------------+
|corr(label, synack)|
+-------------------+
|0.05829892920859981|
+-------------------+



In [12]:
sdf.select(corr('label','dbytes')).show()

+--------------------+
| corr(label, dbytes)|
+--------------------+
|-0.07687086056976948|
+--------------------+



In [13]:
sdf.select(corr('label','Sttl')).show()

+------------------+
| corr(label, Sttl)|
+------------------+
|0.6927414405810921|
+------------------+



In [14]:
sdf.select(corr('label','ct_srv_src')).show()

+-----------------------+
|corr(label, ct_srv_src)|
+-----------------------+
|    0.22904392983552418|
+-----------------------+



In [15]:
sdf.select(corr('label','Dttl')).show()

+-------------------+
|  corr(label, Dttl)|
+-------------------+
|0.09504860499575536|
+-------------------+



In [16]:
sdf.select(corr('synack','dbytes')).show()

+--------------------+
|corr(synack, dbytes)|
+--------------------+
|1.014482864049306...|
+--------------------+



In [17]:
sdf.select(corr('synack','Sttl')).show()

+--------------------+
|  corr(synack, Sttl)|
+--------------------+
|0.042589758081623796|
+--------------------+



In [18]:
sdf.select(corr('synack','ct_srv_src')).show()

+------------------------+
|corr(synack, ct_srv_src)|
+------------------------+
|    -0.27627475063298373|
+------------------------+



In [19]:
sdf.select(corr('synack','Dttl')).show()

+------------------+
|corr(synack, Dttl)|
+------------------+
|0.7469996936368721|
+------------------+



In [20]:
sdf.select(corr('dbytes','Sttl')).show()

+-------------------+
| corr(dbytes, Sttl)|
+-------------------+
|-0.1355151870099393|
+-------------------+



In [21]:
sdf.select(corr('dbytes','Dttl')).show()

+--------------------+
|  corr(dbytes, Dttl)|
+--------------------+
|0.023558872720887036|
+--------------------+



In [22]:
sdf.select(corr('ct_srv_src','Dttl')).show()

+----------------------+
|corr(ct_srv_src, Dttl)|
+----------------------+
|   -0.4173378502004274|
+----------------------+



In [23]:
sdf.select(corr('ct_srv_src','Sttl')).show()

+----------------------+
|corr(ct_srv_src, Sttl)|
+----------------------+
|   0.34607909452631314|
+----------------------+



In [24]:
sdf.select(corr('dbytes','Dttl')).show()

+--------------------+
|  corr(dbytes, Dttl)|
+--------------------+
|0.023558872720887036|
+--------------------+



In [25]:
sdf.select(corr('dbytes','Dttl')).show()

+--------------------+
|  corr(dbytes, Dttl)|
+--------------------+
|0.023558872720887036|
+--------------------+



In [26]:
sdf.crosstab('label','attack_cat').show()

+----------------+--------+--------+-----+--------+-------+-------+------+--------------+---------+-----+
|label_attack_cat|Analysis|Backdoor|  DoS|Exploits|Fuzzers|Generic|Normal|Reconnaissance|Shellcode|Worms|
+----------------+--------+--------+-----+--------+-------+-------+------+--------------+---------+-----+
|               1|    2000|    1746|12264|   33393|  18184|  40000|     0|         10491|     1133|  130|
|               0|       0|       0|    0|       0|      0|      0| 56000|             0|        0|    0|
+----------------+--------+--------+-----+--------+-------+-------+------+--------------+---------+-----+



In [27]:
sdf.select('State','dbytes','Sttl','ct_srv_src','Dttl','synack').show()

+-----+------+----+----------+----+--------------------+
|State|dbytes|Sttl|ct_srv_src|Dttl|              synack|
+-----+------+----+----------+----+--------------------+
|  FIN|   172| 252|         1| 254|                 0.0|
|  FIN| 42014|  62|        43| 252|                 0.0|
|  FIN| 13186|  62|         7| 252|0.061458000000000006|
|  FIN|   770|  62|         1| 252|                 0.0|
|  FIN|   268| 254|        43| 252|            0.071147|
|  FIN|   268| 254|        43| 252| 0.11933099999999999|
|  FIN|   354| 254|        43| 252|            0.069136|
|  FIN|   354| 254|        43| 252|0.059195000000000005|
|  FIN|   354| 254|        43| 252|            0.066133|
|  FIN|   268| 254|        43| 252| 0.06311599999999999|
|  FIN|   268| 254|        11| 252|            0.036508|
|  FIN|  2212|  62|         2| 252|0.052851999999999996|
|  FIN|   268| 254|        43| 252|            0.094537|
|  FIN|   354| 254|        11| 252|             0.07516|
|  FIN|   354| 254|        43| 